In [1]:
import pandas as pd
import numpy as np
import researchpy # pip install researchpy
import re
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)
import random
from scipy import stats # for later on plotting best fit line
from math import ceil 

In [2]:
# Calculated Methodology 

# Average latency of the same bulletins
# DO NOT remove 0 values


In [3]:
# For CVE characteristics!

# Import CVE Bulletin Data
df1=pd.read_csv('../data/cleaned_bulletins.csv')
df1.bulletin=df1.bulletin.str.replace(r"\.txt","")
df1.bulletin=df1.bulletin.apply(lambda x: str(x[:-2])+"01") # Normalize bulletins
df1.CVE_Classification=df1.CVE_Classification.str.replace("Media Framework","Media framework") # Text cleanup
df1=df1[df1.CVE_Classification!="Qualcomm closed-source components 2014-2016 cumulative update"] # Text cleanup
df1=df1[df1.bulletin!="2020-01-01"]
df1=df1[df1.bulletin!="2020-02-01"]
df1=df1[df1.bulletin!="2020-03-01"]

print(len(df1.bulletin.unique()))

# Import carrier data
carrierData=pd.read_csv('../data/allCarrierData_final.csv')
carrierData=carrierData[carrierData.checker==1]
carrierData.drop(columns=['Android_Level_Guessed','checker'],inplace=True)
carrierData['Bulletin_Year']=carrierData.Bulletin_Level
carrierData.Bulletin_Year=carrierData.Bulletin_Year.apply(lambda x: x[:4])
carrierData.Bulletin_Level=carrierData.Bulletin_Level.apply(lambda x: x[:10])

#######################################################
############ Creating Necessary Dataframes ############
#######################################################
print("-----------------------------------Instantiated Variables-----------------------------------------")
severityLevels=df1.Severity.unique()
severityLevels=columns=['High','Critical','Moderate','Low']
#typeLevels defined below...
carrierUniques=carrierData.Carrier.unique()
manufactureUniques=carrierData.Manufacture.unique()
allBulletins=df1.bulletin.unique()

print("-----------------------------------CVE Severity Counts------------------------------------------")
toDataFrame=[]
columns=['Bulletin','Bulletin_Year','High','Critical','Moderate','Low']
for bull in allBulletins:
    tempDF=df1[df1.bulletin==bull]
    tempVals=[bull,bull[:4]]
    for severity in severityLevels:
        check=tempDF[tempDF.Severity==severity]
        tempVals.append(len(check))
    toDataFrame.append(tempVals.copy())

severityCounts=pd.DataFrame(toDataFrame,columns=columns)

print("----------------------------------CVE Type Counts-------------------------------------------")

toDataFrame=[]
columns=['Bulletin','Bulletin_Year','EoP','ID','RCE','DoS']
temps=df1.dropna(subset=['Type'])
typeVals=temps.Type.unique()
for bull in allBulletins:
    tempDF=df1[df1.bulletin==bull]
    tempVals=[bull,bull[:4]]
    for types in typeVals:
        check=tempDF[tempDF.Type==types]
        tempVals.append(len(check))
    toDataFrame.append(tempVals.copy())

typeCounts=pd.DataFrame(toDataFrame,columns=columns)
typeLevels=typeVals

print("------------------------------------CVE Classificiation Counts-----------------------------------------")
toDataFrame=[]
columns=['Bulletin','Bulletin_Year']
categoryVals=df1.CVE_Classification.unique()
columns.extend(categoryVals)

for bull in allBulletins:
    tempDF=df1[df1.bulletin==bull]
    tempVals=[bull,bull[:4]]
    for cat in categoryVals:
        check=tempDF[tempDF.CVE_Classification==cat]
        tempVals.append(len(check))
    toDataFrame.append(tempVals.copy())
categoryCounts=pd.DataFrame(toDataFrame,columns=columns) #create dataframe!

# sum all the columns and remove low counts of CVEs for categories
summation=categoryCounts.sum(axis=0,skipna=True)
summation=summation.iloc[2:] #remove bulletin info
categoryVals=[]
toDropVals=[]
for index,val in summation.iteritems(): #REMOVE CATEGORIES WITH LESS THAN 5 CVES!!!
    if int(val)>=5:
        categoryVals.append(index)
    else:
        toDropVals.append(index)
        
# drop all columns with those low counts
categoryCounts.drop(columns=toDropVals,inplace=True)


print("---------------------------------CVE Severity Coeff--------------------------------------------")

carrierDataCopy=carrierData.copy()
print(carrierDataCopy.columns)
carrierDataCopy.columns=['Carrier', 'Manufacture', 'Model', 'Release_Date', 'Bulletin', 'difference', 'build','Year_Phone_Release', 'Month_Phone_Release','Bulletin_Year']
carrierDataCopy.drop(columns=['Release_Date','Bulletin_Year'],inplace=True)
carrierDataCopy.Bulletin=carrierDataCopy.Bulletin.astype('str')
severityCounts.Bulletin=severityCounts.Bulletin.astype('str')
severityCoeff = pd.merge(carrierDataCopy, severityCounts, on=['Bulletin'],how='outer')
severityCoeff.High=severityCoeff.High.astype('str')
severityCoeff=severityCoeff[severityCoeff.High!='nan']
severityCoeff.High=severityCoeff.High.astype('float64')
severityCoeff.Critical=severityCoeff.Critical.astype('float64')
severityCoeff.Moderate=severityCoeff.Moderate.astype('float64')
severityCoeff.Low=severityCoeff.Low.astype('float64')


print("------------------------------------CVE Type Coeff-----------------------------------------")

carrierDataCopy=carrierData.copy()
carrierDataCopy.columns=['Carrier', 'Manufacture', 'Model', 'Release_Date', 'Bulletin', 'difference', 'build', 'Year_Phone_Release', 'Month_Phone_Release', 'Bulletin_Year']
carrierDataCopy.drop(columns=['Release_Date','Bulletin_Year','Year_Phone_Release', 'Month_Phone_Release'],inplace=True)
carrierDataCopy.Bulletin=carrierDataCopy.Bulletin.astype('str')
typeCounts.Bulletin=typeCounts.Bulletin.astype('str')
typeCoeff = pd.merge(carrierDataCopy, typeCounts, on=['Bulletin'],how='outer')
typeCoeff.EoP=typeCoeff.EoP.astype('str')
typeCoeff=typeCoeff[typeCoeff.EoP!='nan']
typeCoeff.EoP=typeCoeff.EoP.astype('float64')
typeCoeff.ID=typeCoeff.ID.astype('float64')
typeCoeff.RCE=typeCoeff.RCE.astype('float64')
typeCoeff.DoS=typeCoeff.DoS.astype('float64')

print("-----------------------------------CVE Classification Coeff------------------------------------------")

carrierDataCopy=carrierData.copy()
carrierDataCopy.columns=['Carrier', 'Manufacture', 'Model', 'Release_Date', 'Bulletin', 'difference', 'build', 'Year_Phone_Release', 'Month_Phone_Release', 'Bulletin_Year']
carrierDataCopy.drop(columns=['Release_Date','Bulletin_Year','Year_Phone_Release'],inplace=True)
carrierDataCopy.Bulletin=carrierDataCopy.Bulletin.astype('str')
categoryCounts.Bulletin=categoryCounts.Bulletin.astype('str')
catCoeff = pd.merge(carrierDataCopy, categoryCounts, on=['Bulletin'],how='outer')
catCoeff.Framework=catCoeff.Framework.astype('str')
catCoeff=catCoeff[catCoeff.Framework!='nan']

#for each column not bulletin or year...
for col in catCoeff.columns:
    if col != 'Bulletin' and col != 'Bulletin_Year' and col!= 'Carrier' and col!= 'Manufacture'and col!= 'Model' and col != 'Bulletin' and col !="build":
        catCoeff[col]=catCoeff[col].astype('float64')


print("-----------------------------------Total CVE Counts for Bulletins-----------------------------------------")

totalCount=[]
for bull in allBulletins:
    tempDF=df1[df1.bulletin==bull]
    totalCount.append([bull,len(tempDF)])

totalCountDF=pd.DataFrame(totalCount,columns=['Bulletin','Count'])

54
-----------------------------------Instantiated Variables-----------------------------------------
-----------------------------------CVE Severity Counts------------------------------------------
----------------------------------CVE Type Counts-------------------------------------------
------------------------------------CVE Classificiation Counts-----------------------------------------
---------------------------------CVE Severity Coeff--------------------------------------------
Index(['Carrier', 'Manufacture', 'Model', 'Release_Date', 'Bulletin_Level',
       'difference', 'build', 'Year_Phone_Release', 'Month_Phone_Release',
       'Bulletin_Year'],
      dtype='object')
------------------------------------CVE Type Coeff-----------------------------------------
-----------------------------------CVE Classification Coeff------------------------------------------
-----------------------------------Total CVE Counts for Bulletins-----------------------------------------


In [4]:
# for each Sytem category, count number of critical'
# print(carrierData.head())
# print(df1.head())
carrierData['Bulletin']=carrierData.Bulletin_Level.copy()
df1['Bulletin']=df1.bulletin.copy()
checkMe=pd.merge(carrierDataCopy, df1, on=['Bulletin'],how='outer')
print(checkMe.head())

checkSec=checkMe[checkMe.CVE_Classification=="System"]
print(len(checkMe))
print(len(checkSec))
checkSec=checkSec[checkSec.Severity=="Critical"]
print(len(checkSec))



# print(len(df1)) 
# checkSec=df1[df1.CVE_Classification=="System"]
# print(len(checkSec))
# checkSec=checkSec[checkSec.Severity=="Critical"]
# print(len(checkSec))

  Carrier Manufacture               Model    Bulletin  difference  \
0    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-01         2.0   
1    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-01         2.0   
2    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-01         2.0   
3    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-01         2.0   
4    AT&T     Alcatel  Alcatel IdealXCITE  2018-04-01         2.0   

                 build  Month_Phone_Release             CVE Type  Severity  \
0  nan YB4UA40File nan                  6.0  CVE-2017-13274  EoP      High   
1  nan YB4UA40File nan                  6.0  CVE-2017-13275   ID      High   
2  nan YB4UA40File nan                  6.0  CVE-2017-13276  RCE  Critical   
3  nan YB4UA40File nan                  6.0  CVE-2017-13277  RCE  Critical   
4  nan YB4UA40File nan                  6.0  CVE-2017-13278  EoP      High   

                     Updated_AOSP_versions CVE_Classification    bulletin  \
0  6.0, 6.0.1, 7.0, 7.1.1, 7.1.2, 8.0, 

In [5]:
######################################################################################################################
##################################### CVE Severity and Average Carriers ##############################################
######################################################################################################################
 
CoeffDF2=[]
for sev in severityLevels:
    data=[]
    for carI in range(4):
        tempVals2=severityCoeff[severityCoeff.Carrier==carrierUniques[carI]]
        toGraphx=[]
        toGraphy=[]
        for bul in tempVals2.Bulletin.unique():
            tempVals=tempVals2[tempVals2.Bulletin==bul]
            #if tempVals[sev].values[0]!=0: # if we want to remove 0 values
            meanVal=np.mean(tempVals.difference.values)
            someValue=tempVals[sev].values[0]
            CoeffDF2.append([sev,carrierUniques[carI],meanVal,someValue])

CoeffDF2_new=pd.DataFrame(CoeffDF2,columns=['Severity','Carrier','Difference','Severity_Counts'])
pd.set_option('mode.chained_assignment', None)
print("severity;carrier;r-value;p-value;n(points)")
for sev in severityLevels:
    coefTemp=CoeffDF2_new[CoeffDF2_new.Severity==sev]
    for crr in coefTemp.Carrier.unique():
        coefTemp2=coefTemp[coefTemp.Carrier==crr]
        coefTemp2.drop(columns=['Severity','Carrier'],inplace=True)
        coefTemp2=coefTemp2.astype('float')
        coefTemp2.columns=['Difference',str(crr)]
        dfTemp=researchpy.correlation.corr_pair(coefTemp2,"spearman")
        print(str(sev+ ";" +crr + ";"+  dfTemp.iloc[0][0]+ ";" + dfTemp.iloc[0][1]+ ";") + str(dfTemp.iloc[0][2]))#.iloc[0:13,:]
    print("---------------------")
pd.reset_option('mode.chained_assignment')

severity;carrier;r-value;p-value;n(points)
High;AT&T;-0.0940;0.5440;44
High;Sprint;-0.0175;0.9051;49
High;T-Mobile;0.0637;0.6507;53
High;Verizon;0.0567;0.7148;44
---------------------
Critical;AT&T;-0.2999;0.0479;44
Critical;Sprint;-0.2032;0.1614;49
Critical;T-Mobile;-0.3747;0.0057;53
Critical;Verizon;0.0494;0.7501;44
---------------------
Moderate;AT&T;0.0144;0.9260;44
Moderate;Sprint;-0.2912;0.0423;49
Moderate;T-Mobile;-0.3378;0.0134;53
Moderate;Verizon;-0.5323;0.0002;44
---------------------
Low;AT&T;-0.2200;0.1514;44
Low;Sprint;0.0598;0.6833;49
Low;T-Mobile;-0.2328;0.0935;53
Low;Verizon;-0.3807;0.0108;44
---------------------


In [6]:
######################################################################################################################
##################################### CVE Type and Average Carriers ##############################################
######################################################################################################################
CoeffDF2=[]
for typ in typeLevels:
    data=[]
    for carI in range(4):
        tempVals2=typeCoeff[typeCoeff.Carrier==carrierUniques[carI]]
        for bul in tempVals2.Bulletin.unique():
            tempVals=tempVals2[tempVals2.Bulletin==bul]
            #if tempVals[typ].values[0]!=0:
            meanValues=np.mean(tempVals.difference.values)
            someValue=tempVals[typ].values[0]
            CoeffDF2.append([typ,carrierUniques[carI],meanValues,someValue])
            
CoeffDF2_new=pd.DataFrame(CoeffDF2,columns=['Type','Carrier','Difference','Severity_Counts'])
pd.set_option('mode.chained_assignment', None)
print("type;carrier;r-value;p-value;n(points)")
for sev in typeLevels:
    coefTemp=CoeffDF2_new[CoeffDF2_new.Type==sev]
    for crr in coefTemp.Carrier.unique():
        coefTemp2=coefTemp[coefTemp.Carrier==crr]
        coefTemp2.drop(columns=['Type','Carrier'],inplace=True)
        coefTemp2=coefTemp2.astype('float')
        coefTemp2.columns=['Difference',str(crr)]
        dfTemp=researchpy.correlation.corr_pair(coefTemp2,"spearman")
        print(str(sev+ ";" + crr + ";" + dfTemp.iloc[0][0]+ ";" + dfTemp.iloc[0][1]+ ";") + str(dfTemp.iloc[0][2]))#.iloc[0:13,:]
    print("---------------------")
pd.reset_option('mode.chained_assignment')

type;carrier;r-value;p-value;n(points)
EoP;AT&T;0.0897;0.5628;44
EoP;Sprint;0.3027;0.0345;49
EoP;T-Mobile;0.3834;0.0046;53
EoP;Verizon;0.6578;0.0000;44
---------------------
RCE;AT&T;0.1575;0.3074;44
RCE;Sprint;0.1749;0.2293;49
RCE;T-Mobile;0.0316;0.8224;53
RCE;Verizon;0.4152;0.0051;44
---------------------
DoS;AT&T;0.0032;0.9836;44
DoS;Sprint;0.4004;0.0044;49
DoS;T-Mobile;0.4458;0.0008;53
DoS;Verizon;0.3855;0.0098;44
---------------------
ID;AT&T;0.0084;0.9570;44
ID;Sprint;0.2276;0.1158;49
ID;T-Mobile;0.1954;0.1608;53
ID;Verizon;0.5478;0.0001;44
---------------------


In [7]:
print(CoeffDF2_new.head())

  Type Carrier  Difference  Severity_Counts
0  EoP    AT&T   37.000000              7.0
1  EoP    AT&T   38.400000              3.0
2  EoP    AT&T   20.875000              3.0
3  EoP    AT&T   37.142857             10.0
4  EoP    AT&T   12.333333              0.0


In [8]:


#dfTemp=researchpy.correlation.corr_pair(DF)
data=[]
# for each severity, calculate the counts on each bulletin
dfTEMPTEMP=CoeffDF2_new[CoeffDF2_new.Type=="DoS"]

for car in dfTEMPTEMP.Carrier.unique():
    dtmp=dfTEMPTEMP[dfTEMPTEMP.Carrier==car]
    data.append(go.Scatter(
            x=dtmp.Difference.values,
            y=dtmp.Severity_Counts.values,
            mode='markers',
            name=str(car)
        ))

# Edit the layout
layout = dict(title = "Avg Latency of Years after Release",
              yaxis = dict(title = 'Prior Missed Bulletins',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              xaxis = dict(title = 'Avg Latency of Patch'),
              plot_bgcolor='rgba(0,0,0,0)',
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='avg-latency-scatter.png')




In [9]:

######################################################################################################################
##################################### CVE Category and Average Carriers ##############################################
######################################################################################################################
print(len(categoryVals))
print()

#axis values for formatting
axisVals=[["x","y"],["x2","y"],["x","y3"],["x4","y4"]]

CoeffDF2=[]
for cat in categoryVals:
    data=[]
    for carI in range(4):
        tempVals2=catCoeff[catCoeff.Carrier==carrierUniques[carI]]
        for bul in tempVals2.Bulletin.unique():
            tempVals=tempVals2[tempVals2.Bulletin==bul]
            #if tempVals[cat].values[0]!=0:
            meanValues=np.mean(tempVals.difference.values)
            someValue=tempVals[cat].values[0]
            CoeffDF2.append([cat,carrierUniques[carI],meanValues,someValue])
            
CoeffDF2_new=pd.DataFrame(CoeffDF2,columns=['Category','Carrier','Difference','Severity_Counts'])
pd.set_option('mode.chained_assignment', None)
#print(CoeffDF2_new)
print("classification;carrier;r-value;p-value;n(points)")
for sev in categoryVals:
    coefTemp=CoeffDF2_new[CoeffDF2_new.Category==sev]
    for crr in coefTemp.Carrier.unique():
        coefTemp2=coefTemp[coefTemp.Carrier==crr]
        coefTemp2.drop(columns=['Category','Carrier'],inplace=True)
        coefTemp2=coefTemp2.astype('float')
        coefTemp2.columns=['Difference',str(crr)]
        if len(coefTemp2)<=1:
            print(str(crr + ";" +sev+ ";" + "N/A" + ";" + "N/A" + ";" )+str(len(coefTemp2)))#.iloc[0:13,:]
        else:
            dfTemp=researchpy.correlation.corr_pair(coefTemp2,"spearman")
            if float(dfTemp.iloc[0][1])<=0.05:
                print(str(sev+ ";" +crr + ";" +  dfTemp.iloc[0][0]+ ";" + dfTemp.iloc[0][1]+ ";") + str(dfTemp.iloc[0][2]))#.iloc[0:13,:]
    
pd.reset_option('mode.chained_assignment')
            

25

classification;carrier;r-value;p-value;n(points)
Media framework;Sprint;0.3139;0.0281;49
Media framework;Verizon;0.5411;0.0001;44
System;Sprint;0.4310;0.0020;49
System;T-Mobile;0.5626;0.0000;53
System;Verizon;0.4853;0.0008;44
Framework;T-Mobile;0.3843;0.0045;53
Framework;Verizon;0.4333;0.0033;44
Elevation of Privilege Vulnerability in Qualcomm Wi-Fi Driver;AT&T;-0.3215;0.0333;44
Remote Code Execution Vulnerability in Mediaserver;AT&T;-0.4378;0.0030;44
Remote Code Execution Vulnerability in Mediaserver;T-Mobile;-0.3074;0.0252;53
Elevation of Privilege Vulnerability in Mediaserver;AT&T;-0.3215;0.0333;44
Information Disclosure Vulnerability in Mediaserver;AT&T;-0.3228;0.0326;44
Elevation of Privilege Vulnerability in Wi-Fi;AT&T;-0.3354;0.0260;44
Elevation of Privilege Vulnerability in Wi-Fi;Sprint;-0.3364;0.0181;49
Elevation of Privilege Vulnerability in Wi-Fi;T-Mobile;-0.2766;0.0449;53
Denial of service vulnerability in Mediaserver;AT&T;0.2983;0.0492;44
Denial of service vulnerabilit

In [10]:
######################################################################################################################
##################################### CVE Total Counts vs Latency ##############################################
######################################################################################################################

CoeffDF2=[]
#for each bulletin, capture both dataframes...
toGraphx=[]
toGraphy=[]
for bul in totalCountDF.Bulletin.unique():
    tempVals=carrierData[carrierData.Bulletin_Level==bul]
    #if tempVals[sev].values[0]!=0:
    meanVal=np.mean(tempVals.difference.values)
    someVal=totalCountDF[totalCountDF.Bulletin==bul].values[0][1]
    CoeffDF2.append([bul,meanVal,someVal])
    
CoeffDF2_new=pd.DataFrame(CoeffDF2,columns=['Bulletin','Difference','Severity_Counts'])
CoeffDF2_new.drop(columns=['Bulletin'],inplace=True)
pd.set_option('mode.chained_assignment', None)
#print(CoeffDF2_new)
print("r-value;p-value;n(points)")
dfTemp=researchpy.correlation.corr_pair(CoeffDF2_new)
print(str(dfTemp.iloc[0][0]+ ";" + dfTemp.iloc[0][1]+ ";") + str(dfTemp.iloc[0][2]))#.iloc[0:13,:]
      
        
pd.reset_option('mode.chained_assignment')

r-value;p-value;n(points)
-0.2822;0.0406;53


/Users/kjo/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning:

Mean of empty slice.

/Users/kjo/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning:

invalid value encountered in double_scalars



In [11]:
######################################################################################################################
############################### CVE Total Counts vs Latency across Carriers ##########################################
######################################################################################################################
data=[]
CoeffDF2=[]
for carI in range(4):
    #for each bulletin, capture both dataframes...
    tempDF=carrierData[carrierData.Carrier==carrierUniques[carI]]
    for bul in totalCountDF.Bulletin.unique():
        tempVals=tempDF[tempDF.Bulletin_Level==bul]
        if len(tempVals)>0:
            #if tempVals[sev].values[0]!=0:
            meanVal=np.mean(tempVals.difference.values)
            someVal=totalCountDF[totalCountDF.Bulletin==bul].values[0][1]
            CoeffDF2.append([carrierUniques[carI],bul,meanVal,someVal])
    
CoeffDF2_new=pd.DataFrame(CoeffDF2,columns=['Carrier','Bulletin','Difference','CVE_Counts'])
pd.set_option('mode.chained_assignment', None)
print("carrier;r-value;p-value;n(points)")
for crr in CoeffDF2_new.Carrier.unique():
    coefTemp2=CoeffDF2_new[CoeffDF2_new.Carrier==crr]
    #print(str(man)+ " " + str(len(coefTemp2))+ " " +str(coefTemp2.corr(method='pearson').iloc[0,1]))
    coefTemp2.drop(columns=['Bulletin','Carrier'],inplace=True)
    coefTemp2=coefTemp2.astype('float')
    coefTemp2.columns=['Difference',str(crr)]
    dfTemp=researchpy.correlation.corr_pair(coefTemp2)
    print(str(crr + ";" +  dfTemp.iloc[0][0]+ ";" + dfTemp.iloc[0][1]+ ";") + str(dfTemp.iloc[0][2]))#.iloc[0:13,:]
print("---------------------")
pd.reset_option('mode.chained_assignment')

carrier;r-value;p-value;n(points)
AT&T;-0.2230;0.1457;44
Sprint;-0.0795;0.5869;49
T-Mobile;-0.2841;0.0392;53
Verizon;-0.1609;0.2968;44
---------------------


In [12]:
######################################################################################################################
################# Skipped Bulletins or Days since Last Update vs Latency across Carriers #############################
######################################################################################################################

# Now let's count number of bulletins skipped in-between security updates. 
# For each security update [1:] not including the first, count the days between the previous bulletin.

data=[]
carResults=pd.DataFrame()
carrierData.Bulletin_Level=pd.to_datetime(carrierData.Bulletin_Level)
carrierData.Release_Date=pd.to_datetime(carrierData.Release_Date)

carrierData=carrierData.sort_values(by=['Carrier','Manufacture','Model','Bulletin_Level','Bulletin_Level','Release_Date'])

for car in carrierData.Carrier.unique():
    specCar=carrierData[carrierData.Carrier==car]
    for man in specCar.Manufacture.unique():
        specMan=specCar[specCar.Manufacture==man]
        for model in specMan.Model.unique():
            specMod=specMan[specMan.Model==model]
            if len(specMod)>1:
                df1st=specMod.iloc[1:,:].copy()
                df2nd=specMod.iloc[:-1,:].copy()
                
                df1st.reset_index(inplace=True)
                df2nd.reset_index(inplace=True)

                df1st['prior_missed_buls']=df1st.Bulletin_Level-df2nd.Bulletin_Level
                df1st.prior_missed_buls=df1st.prior_missed_buls.apply(lambda x: round(int(x.days)/29)-1) # convert dattime days to integers
                
                df1st['days_since_last_update']=df1st.Release_Date-df2nd.Release_Date
                df1st.days_since_last_update=df1st.days_since_last_update.apply(lambda x: x.days) # convert dattime days to integers
                #print(df1st)
                if len(carResults)==0:
                    carResults=df1st.copy()
                else:
                    carResults=carResults.append(df1st.copy())
carResults.reset_index(drop=True,inplace=True)


test=pd.merge(carrierData, carResults,  how='outer', left_on=['Carrier','Manufacture','Model','Release_Date','Bulletin_Level'], right_on = ['Carrier','Manufacture','Model','Release_Date','Bulletin_Level'])
test.drop(columns=['index','difference_y','Year_Phone_Release_y','Month_Phone_Release_y', 'Bulletin_Year_y','build_y',"Bulletin_y","Bulletin_x"],inplace=True)
print(test.head())
test.columns=['Carrier','Manufacture','Model','Release_Date','Bulletin_Level','difference',' build','Year_Phone_Release','Month_Phone_Release','Bulletin_Year','Prior_Missed_Bulletins','Days_Since_Last_Security_Updates']
print(test)
test.to_csv('../data/allCarrierData_Latest.csv',index=False)


import researchpy
print("------------------- Skipped Bulletins -----------------------")
print("SPEARMAN: Correlation coefficient between latency and number of skipped bulletins")
DF=carResults.drop(columns=['Carrier', 'Manufacture', 'Model', 'Release_Date', 'Bulletin_Level','days_since_last_update','Year_Phone_Release','Month_Phone_Release','Bulletin_Year'])
dfTemp=researchpy.correlation.corr_pair(DF,"spearman")
print(dfTemp)

print("-------------------- Days Since Previous Update ----------------------")
print("SPEARMAN: Correlation coefficient between latency and number of skipped bulletins")
DF=carResults.drop(columns=['Carrier', 'Manufacture', 'Model','Release_Date', 'Bulletin_Level','prior_missed_buls','Year_Phone_Release','Month_Phone_Release','Bulletin_Year'])
dfTemp=researchpy.correlation.corr_pair(DF,"spearman")
# print(dfTemp)



  Carrier Manufacture               Model Release_Date Bulletin_Level  \
0    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-04-01   
1    AT&T     Alcatel  Alcatel IdealXCITE   2018-10-24     2018-08-01   
2    AT&T  BlackBerry   BlackBerry KEYone   2018-05-29     2018-04-01   
3    AT&T  BlackBerry   BlackBerry KEYone   2018-06-12     2018-06-01   
4    AT&T  BlackBerry   BlackBerry KEYone   2018-09-24     2018-09-01   

   difference_x                                            build_x  \
0             2                                nan YB4UA40File nan   
1            84        JO.2.0.c1-00267-8909_GEN_PACK-2 nan YAYUAY0   
2            58  MPSS.TA.2.3.c1-00723-8953_GEN_PACK-1.153374.2 ...   
3            11  MPSS.TA.2.3.c1-00723-8953_GEN_PACK-1.153374.2 ...   
4            23  MPSS.TA.2.3.c1-00723-8953_GEN_PACK-1.153374.2 ...   

   Year_Phone_Release_x  Month_Phone_Release_x Bulletin_Year_x  \
0                  2017                      6            2018   
1       

In [13]:
# For each update, calculate the difference between the carrier release date and phone publish date
print(DF.head())

   index  difference                                              build  \
0      2          84        JO.2.0.c1-00267-8909_GEN_PACK-2 nan YAYUAY0   
1     11          11  MPSS.TA.2.3.c1-00723-8953_GEN_PACK-1.153374.2 ...   
2     13          23  MPSS.TA.2.3.c1-00723-8953_GEN_PACK-1.153374.2 ...   
3     22          42  01.01_U11440801_96.01.51207A_F 3045450 3.38.50...   
4     23          35         01.01_U11440801_96.01.51207A_F 3046689 nan   

     Bulletin  days_since_last_update  
0  2018-08-01                     204  
1  2018-06-01                      14  
2  2018-09-01                     104  
3  2016-08-01                     329  
4  2016-09-01                      24  


In [14]:

# carResults.head()

In [15]:
DF=carResults.drop(columns=['Carrier', 'Manufacture', 'Model', 'Release_Date', 'Bulletin_Level','Year_Phone_Release','Month_Phone_Release','days_since_last_update'])
#dfTemp=researchpy.correlation.corr_pair(DF)
data=[]
# for each severity, calculate the counts on each bulletin


data.append(go.Scatter(
        x=DF.difference.values,
        y=DF.prior_missed_buls.values,
        mode='markers',
    ))

# Edit the layout
layout = dict(title = "Avg Latency of Years after Release",
              yaxis = dict(title = 'Prior Missed Bulletins',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              xaxis = dict(title = 'Avg Latency of Patch'),
              plot_bgcolor='rgba(0,0,0,0)',
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='avg-latency-scatter.png')


In [16]:
DF=carResults.drop(columns=['Carrier', 'Manufacture', 'Model','Release_Date', 'Bulletin_Level','Year_Phone_Release','Month_Phone_Release','prior_missed_buls'])
#dfTemp=researchpy.correlation.corr_pair(DF)
data=[]
# for each severity, calculate the counts on each bulletin


data.append(go.Scatter(
        x=DF.difference.values,
        y=DF.days_since_last_update.values,
        mode='markers',
    ))

# Edit the layout
layout = dict(title = "Avg Latency of Years after Release",
              yaxis = dict(title = 'Days from previous update to current',showgrid=True, gridcolor='rgb(219, 219, 219)'),
              xaxis = dict(title = 'Avg Latency of Patch'),
              plot_bgcolor='rgba(0,0,0,0)',
              )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='avg-latency-scatter.png')
